In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Missing Data').getOrCreate()

In [7]:
df = spark.read.csv('ContainsNull.csv', inferSchema=True, header=True)

In [8]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [9]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [10]:
# We will handle missing data in this notebook. There are three ways to handle missing data:
# 1. Drop the row
# 2. Keep the data
# 3. Replace null values with someother values

In [13]:
# DROPPING MISSING VALUES
df.na.drop().show() # so if we delete na values it will delete all rows which contain atleast one null

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [14]:
df.show() # you can still see the column as drop is not a inplace function.

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [16]:
# We can also define a threshold value
df.na.drop(thresh=2).show() # rows with 2 or more null values will be deleted

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [17]:
# There is one more parameter "how". default value is any. This means if row has any null value
# then row willbe deleted.
df.na.drop(how='any').show() # in this case you dont have to mention how.

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [18]:
# You can also assign 'all' value to how param.
df.na.drop(how='all').show() # Delete only if all column values are null in row
# So here every column has atleast Id so no rows was deleted.

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [20]:
# you can also check missing values for a particular colum
df.na.drop(subset=['Sales']).show() # so here we can see Name is null but it doesnt matter
#  it will only delete if Sales value is null

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [21]:
# Fill missing values

In [22]:
df.na.fill('MISSING_VALUES', subset=['Name']).show() # it will fill MISSING_VALUES text to Name

+----+--------------+-----+
|  Id|          Name|Sales|
+----+--------------+-----+
|emp1|          John| null|
|emp2|MISSING_VALUES| null|
|emp3|MISSING_VALUES|345.0|
|emp4|         Cindy|456.0|
+----+--------------+-----+



In [25]:
# lets replace null values in Sales to mean value
mean_data = df.agg({'Sales': 'mean'}).collect()

In [28]:
mean_sales = mean_data[0][0]

In [29]:
mean_sales

400.5

In [31]:
df.na.fill(mean_sales, subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

